In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torchvision.transforms as transforms
# import torchvision.models as models
import torch.optim as optim

In [11]:

class ConvEncoder(nn.Module):
  """ create convolutional layers to extract features
  from input multipe spectral images
  
  Attributes:
  data : input data to be encoded
  """

  def __init__(self, in_channel):
      super(ConvEncoder,self).__init__()
      #Convolution 1
      self.conv1=nn.Conv2d(in_channels=in_channel,out_channels=128, kernel_size=7, stride=2, padding = 3)   # padding='valid', there's no padding; padding='same' the input are zero-padded
      # nn.init.xavier_uniform(self.conv2.weight)

      #Convolution 2
      self.conv2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=1, padding='same')
      
      #Convolution 3
      self.conv3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding='same')
      self.bn3 = nn.BatchNorm2d(128)

      #Convolution 4      
      self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=1, padding='same')
      self.bn4 = nn.BatchNorm2d(256)

      #Convolution 5      
      self.conv5 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1, padding='same')
      self.bn4 = nn.BatchNorm2d(512)

      # activation function
      self.relu= nn.ReLU()

      #Max Pool 1
      self.maxpool = nn.MaxPool2d(kernel_size=2,return_indices=True)

      #Average Pool 1
      # self.averagepool = nn.AvgPool2d(kernel_size=2,return_indices=True)
      # self.maxpool1= nn.MaxPool2d(kernel_size=2,return_indices=True)

  def forward(self,x):

      pool_info = []

      out=self.relu(self.conv1(x))
      # out=self.relu(self.conv2(out))
      # out=self.bn3(out)

      size1 = out.size()
      out,indices1=self.maxpool(out)
      pool_info.append([indices1,size1])

      out=self.relu(self.conv3(out))
      out=self.relu(self.conv4(out))
      # out=self.bn4(out)
      
      size2 = out.size()
      out,indices2=self.maxpool(out)
      pool_info.append([indices2,size2])

      # out=self.conv5(out)

      return(out, pool_info)

In [12]:

class DeConvDecoder(nn.Module):
  """ 
  reconstruct image from extracted features
  
  Attributes:
  features : input data to be encoded
  in_channel: reconstructed channels
  """
  def __init__(self, in_channel):
      super(DeConvDecoder,self).__init__()

      #De Convolution 1
      self.deconv1=nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=1)
      # nn.init.xavier_uniform(self.deconv1.weight)
      # self.swish4=nn.ReLU()
      self.bn1 = nn.BatchNorm2d(512)

      #De Convolution 2
      self.deconv2=nn.ConvTranspose2d(in_channels=256,out_channels=128,kernel_size=1)
      self.bn2 = nn.BatchNorm2d(256)

      #De Convolution 3
      self.deconv3=nn.ConvTranspose2d(in_channels=128,out_channels=128,kernel_size=3, padding = 1)
      self.bn3 = nn.BatchNorm2d(128)

      #De Convolution 4
      self.deconv4=nn.ConvTranspose2d(in_channels=128,out_channels=128,kernel_size=1)
      self.bn4 = nn.BatchNorm2d(128)

      #DeConvolution 5
      self.deconv5=nn.ConvTranspose2d(in_channels=128,out_channels=in_channel,kernel_size=7, stride=2, padding = 3)
      # nn.init.xavier_uniform(self.deconv3.weight)
      # self.swish6=nn.ReLU()

      # activation function
      self.relu= nn.ReLU()

      #Max UnPool 1
      self.maxunpool=nn.MaxUnpool2d(kernel_size=2)
      #Max UnPool 2
      # self.maxunpool2=nn.MaxUnpool2d(kernel_size=2)


  def forward(self,x, pool_info):

      # out=self.relu(self.deconv1(x))

      indices2,size2 = pool_info[1]
      # out=self.maxunpool(x)
      out=self.maxunpool(x,indices2,size2)

      out=self.relu(self.deconv2(out))
      out=self.relu(self.deconv3(out))
      # out=self.bn3(out)

      indices1,size1 = pool_info[0]
      # out=self.maxunpool(out)
      out=self.maxunpool(out,indices1,size1)
      
      # out=self.relu(self.deconv4(out))
      out=self.deconv5(out)
      # out=self.swish6(out)

      return(out)

In [13]:
class EncoderDecoder(nn.Module):
  """ create convolutional layers to extract features
  from input multipe spectral images
  
  Attributes:
  data : input data to be encoded
  """

  def __init__(self, in_channel):
      super(EncoderDecoder,self).__init__()
      #Convolution 1
      self.conv1=nn.Conv2d(in_channels=in_channel,out_channels=128, kernel_size=7, stride=2, padding = 3)   # padding='valid', there's no padding; padding='same' the input are zero-padded
      # nn.init.xavier_uniform(self.conv2.weight)
      self.relu= nn.ReLU()
      #Convolution 2
      self.conv2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=1, padding='same')

      #Max Pool 1
      self.maxpool = nn.MaxPool2d(kernel_size=2,return_indices=True)

      #Convolution 3
      self.conv3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding='same')

      #Convolution 4      
      self.conv4 = nn.Conv2d(in_channels=128, out_channels=512, kernel_size=1, padding='same')

      #Average Pool 1
      # self.averagepool = nn.AvgPool2d(kernel_size=2,return_indices=True)
      # self.maxpool1= nn.MaxPool2d(kernel_size=2,return_indices=True)

      #Convolution 5      
      self.conv5 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1, padding='same')

      
      #De Convolution 5
      self.deconv5=nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=1)
      # nn.init.xavier_uniform(self.deconv1.weight)
      # self.swish4=nn.ReLU()
      #Max UnPool 1
      self.maxunpool=nn.MaxUnpool2d(kernel_size=2)

      #De Convolution 4
      self.deconv4=nn.ConvTranspose2d(in_channels=512,out_channels=128,kernel_size=1)

      #De Convolution 3
      self.deconv3=nn.ConvTranspose2d(in_channels=128,out_channels=128,kernel_size=3, padding = 1)

      #Max UnPool 2
      # self.maxunpool2=nn.MaxUnpool2d(kernel_size=2)

      #De Convolution 2
      self.deconv2=nn.ConvTranspose2d(in_channels=128,out_channels=128,kernel_size=1)

      #DeConvolution 1
      self.deconv1=nn.ConvTranspose2d(in_channels=128,out_channels=in_channel,kernel_size=7, stride=2, padding = 3)
      # nn.init.xavier_uniform(self.deconv3.weight)
      # self.swish6=nn.ReLU()
      
      
  def forward(self,x):
      # pool_info = []
      out=self.relu(self.conv1(x))
      # out,indices1=self.maxpool(out)
      out=self.relu(self.conv2(out))
      size1 = out.size()
      out,indices1=self.maxpool(out)
      # pool_info.append([indices1,size1])
      # out,indices2=self.maxpool(out)
      out=self.relu(self.conv3(out))
      out=self.relu(self.conv4(out))
      # out,indices2=self.averagepool(out)
      size2 = out.size()
      out,indices2=self.maxpool(out)
      
      # pool_info.append([indices2,size2])
      # out=self.conv5(out)
      # print(pool_info)
        # out=self.relu(self.deconv5(x))
      # indices2,size2 = pool_info[1]
      # out=self.maxunpool(x)
      out=self.maxunpool(out,indices2)
      out=self.relu(self.deconv4(out))
      out=self.relu(self.deconv3(out))
      # indices1,size1 = pool_info[0]
      # out=self.maxunpool(out)
      out=self.maxunpool(out,indices1)
      # out=self.maxunpool2(out,indices1,size1)
      out=self.relu(self.deconv2(out))
      out=self.deconv1(out)
      # out=self.swish6(out)
      return(out)

In [14]:
pool = nn.MaxPool2d(2, stride=2, return_indices=True)
unpool = nn.MaxUnpool2d(2, stride=2)
input = torch.tensor([[[[ 1.,  2.,  3.,  4.],
                        [5., 6., 7., 8.,],
                        [ 11.,  12.,  13.,  14.],
                        [15., 16., 17., 18.]]]])
print(input.size())
output, indices = pool(input)
print(output.size())
unpool(output, indices)
# # Now using output_size to resolve an ambiguous size for the inverse
# input = torch.torch.tensor([[[[ 1.,  2.,  3., 4., 5.],
# output, indices = pool(input)
# # This call will not work without specifying output_size
# unpool(output, indices, output_size=input.size())

torch.Size([1, 1, 4, 4])
torch.Size([1, 1, 2, 2])


tensor([[[[ 0.,  0.,  0.,  0.],
          [ 0.,  6.,  0.,  8.],
          [ 0.,  0.,  0.,  0.],
          [ 0., 16.,  0., 18.]]]])

In [ ]:
print(indices)

tensor([[[[ 5,  7],
          [13, 15]]]])


In [ ]:
!wget -q  https://share.phys.ethz.ch/~pf/albecker/abc/09072022_1154_train.h5
!wget -q  https://share.phys.ethz.ch/~pf/albecker/abc/09072022_1154_val.h5
!wget -q https://share.phys.ethz.ch/~pf/albecker/abc/09072022_1154_test.h5

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from abcdataset import get_h5_images
from abcdataset import ABCDataset
from visualization import ShowBandsCombination, plot_spatial

In [15]:

# <KeysViewHDF5 ['agbd', 'cloud', 'images', 'lat', 'lon', 'scl']>
keys = ['agbd', 'images', 'cloud', 'lat', 'lon', 'scl']
data_type = np.float64
cat_vi = True
cat_cloud = True
cat_coord = True
cat_scl = True

train_file = "09072022_1154_train.h5"
validate_file = "09072022_1154_val.h5"
test_file = "09072022_1154_test.h5"

train_biomasses, train_images = get_h5_images(train_file, keys, data_type, cat_vi, cat_cloud, cat_coord, cat_scl)
validate_biomasses,validate_images = get_h5_images(validate_file, keys, data_type, cat_vi, cat_cloud, cat_coord, cat_scl)
test_biomasses,test_images = get_h5_images(test_file, keys, data_type, cat_vi, cat_cloud, cat_coord, cat_scl)

In [ ]:
max = train_images.max((0,2,3))
min = train_images.min((0,2,3))

In [ ]:
# showimage = ShowBandsCombination(train_images[0:8,1:11,:], train_biomasses[0:8], [max,0])
# showimage.imshow_all_single()
# showimage.imshow_selectBands_3bands([2,3,4,5,6,7])
# showimage.imshow_selectBands_3bands([[3,2,1],[7,3,2],[7,6,3],[9,8,6]])
# imshow_chw(train_images[233,0:11,:])

In [ ]:
# plot_spatial(train_images[0:2,:], train_biomasses[0:2])

In [16]:
validate_biomasses2 = np.log(validate_biomasses)
train_biomasses2 = np.maximum(np.log(train_biomasses),0)
test_biomasses2 = np.log(test_biomasses)

In [18]:
validate_biomasses1 = validate_biomasses/500
train_biomasses1 = np.minimum(train_biomasses/500,1)
test_biomasses1 = test_biomasses/500

In [19]:
train_test = train_images[:,0:15, :,:]
validate_test = validate_images[:,0:15,:,:]
test_test = test_images[:,0:15,:,:]


mean = train_test.mean((0,2,3))
std = train_test.std((0,2,3))

# Load and preprocess your data as required

batch_size = 16

train_dataset = ABCDataset(train_test, train_biomasses1, mean, std)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

validate_dataset = ABCDataset(validate_test, validate_biomasses1, mean, std)
validate_loader = torch.utils.data.DataLoader(validate_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

test_dataset = ABCDataset(test_test, test_biomasses1, mean, std)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [21]:
from torchsummary import summary

in_channel = train_test.shape[1]


conv = ConvEncoder(in_channel)
summary(conv, (15, 15, 15))
deconv = DeConvDecoder(in_channel)
# summary(deconv, (512, 2, 2))
# conv_deconv = EncoderDecoder(in_channel)
# summary(conv_deconv, (10, 15, 15))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 128, 8, 8]          94,208
              ReLU-2            [-1, 128, 8, 8]               0
         MaxPool2d-3  [[-1, 128, 4, 4], [-1, 128, 4, 4]]               0
            Conv2d-4            [-1, 128, 4, 4]         147,584
              ReLU-5            [-1, 128, 4, 4]               0
            Conv2d-6            [-1, 256, 4, 4]          33,024
              ReLU-7            [-1, 256, 4, 4]               0
         MaxPool2d-8  [[-1, 256, 2, 2], [-1, 256, 2, 2]]               0
Total params: 274,816
Trainable params: 274,816
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 39.78
Params size (MB): 1.05
Estimated Total Size (MB): 40.84
----------------------------------------------------------------


In [22]:

criterion = nn.MSELoss()  # Mean Squared Error loss function
optimizer = optim.Adam(list(conv.parameters()) + list(deconv.parameters()), lr=0.001)  # Adam optimizer
# optimizer = optim.Adam(conv.parameters().parameters(), lr=0.001)  # Adam optimizer

In [23]:
def validate_model(conv, deconv, validate_loader,criterion):
    val_loss = 0.0
    # evaluate model at end of epoch
    for i, data in enumerate(validate_loader, 0):
    # Get inputs and labels
        inputs, labels = data
        
        features, pool_size = conv(inputs)
        outputs = deconv(features, pool_size )

        # Compute loss
        loss = criterion(outputs, inputs)
        val_loss +=  loss.item()
        
        return val_loss

In [45]:

num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    train_loss = 0.0
    val_loss = 0.0
    
    for i, data in enumerate(train_loader, 0):
        # Get inputs and labels
        inputs, labels = data
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        # 
        features, pool_size = conv(inputs)
        outputs = deconv(features, pool_size )
         
        # outputs =conv_deconv(inputs)
        # Compute loss
        loss = criterion(outputs, inputs)
        
        # Backward pass and optimization
        loss.backward()
        # update weights
        optimizer.step()
        
        # Print statistics
        running_loss += loss.item()
        train_loss +=  loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss))
            running_loss = 0.0

    val_loss = validate_model(conv, deconv, validate_loader,criterion)
    print(f'the training loss is {train_loss}, the validation loss is {val_loss}')

    



[1,   100] loss: 0.020
[1,   200] loss: 0.019
[1,   300] loss: 0.020
[1,   400] loss: 0.021
[1,   500] loss: 0.019
[1,   600] loss: 0.020
[1,   700] loss: 0.054
[1,   800] loss: 0.024
[1,   900] loss: 0.034
[1,  1000] loss: 0.024
[1,  1100] loss: 0.020
[1,  1200] loss: 0.021
[1,  1300] loss: 0.024
[1,  1400] loss: 0.021
[1,  1500] loss: 0.020
the training loss is 0.3784838331193896, the validation loss is 0.006694563198834658
[2,   100] loss: 0.019
[2,   200] loss: 0.019
[2,   300] loss: 0.022
[2,   400] loss: 0.020
[2,   500] loss: 0.018
[2,   600] loss: 0.029
[2,   700] loss: 0.021
[2,   800] loss: 0.026
[2,   900] loss: 0.021
[2,  1000] loss: 0.019
[2,  1100] loss: 0.020
[2,  1200] loss: 0.021
[2,  1300] loss: 0.021
[2,  1400] loss: 0.020
[2,  1500] loss: 0.019
the training loss is 0.3294147147971671, the validation loss is 0.013803680427372456
[3,   100] loss: 0.018
[3,   200] loss: 0.032
[3,   300] loss: 0.031
[3,   400] loss: 0.030
[3,   500] loss: 0.020
[3,   600] loss: 0.019
[3

In [42]:
def plot_spectrums(images, label):
  """
  plot the spectrum for each pixels in the image
  image is numpy array with shape (C, H,W)
  """
  # img_shp = images.shape
  # n = img_shp[0]
  n = len(images)
  fig, axs = plt.subplots(1, n, figsize=(n*5, 4))
  fig = plt.gcf()
  # fig.set_size_inches(18.5,10.5)
  fig.suptitle(str(label))

  for l in range(n):
    image = images[l]

    c,h,w = image.shape
    img = image.reshape(c,-1)

    for i in range(w*h):
      axs[l].plot(range(c),img[:c,i])
    plt.title(f'The biomass is {label}')
    axs[l].set_xlabel('channels')
    axs[l].set_ylabel('intensity')
    # plt.ylim([0,6000])
    # plt.xlim([0,11])

  plt.subplots_adjust(right=0.8)
  plt.show()
  


def scatter_spectrums(images, label):
  """
  plot the spectrum for each pixels in the image
  image is numpy array with shape (C, H,W)
  """
  # img_shp = images.shape
  # n = img_shp[0]
  n = len(images)
  fig, axs = plt.subplots(1, n, figsize=(n*5, 4))
  fig = plt.gcf()
  # fig.set_size_inches(18.5,10.5)
  fig.suptitle(str(label))

  for l in range(n):
    image = images[l]

    c,h,w = image.shape
    img = image.reshape(c,-1)

    for i in range(w*h):
      axs[l].plot(range(c),img[:c,i], '*')
    plt.title(f'The biomass is {label}')
    axs[l].set_xlabel('channels')
    axs[l].set_ylabel('intensity')
    # plt.ylim([0,6000])
    # plt.xlim([0,11])

  plt.subplots_adjust(right=0.8)
  plt.show()
  

def scatter_spectrum(image, label):
  """
  plot the spectrum for each pixels in the image
  image is numpy array with shape (C, H,W)
  """
  c,h,w = image.shape
  # # assigning coordinates
  # y = np.linspace(1, w*h, w*h)
  # x = np.linspace(1, c, c)
  # X, Y = np.meshgrid(x, y)
  # # reshape image to fit plot
  img = image.reshape(c,-1)

  # Change the Size of Graph using Figsize
  fig = plt.figure(figsize=(6, 5))

  for i in range(w*h):
    # plt.scatter(range(c),img[:c,i])
    plt.plot(range(c),img[:c,i], '*')
  plt.title(f'The biomass is {label}')
  plt.xlabel('channels')
  plt.ylabel('intensity')
  # plt.ylim([0,6000])
  # plt.xlim([0,11])
  plt.show()


In [44]:
from visualization import plot_spectrum
total_loss = 0
total_samples = 0
for data in test_loader:
    inputs, labels = data
    # outputs =conv_deconv(inputs)
    features, pool_size = conv(inputs)
    outputs = deconv(features, pool_size )
      
    for l in range(10):

        # imgi = inputs[l].permute(1,2,0) 
        imgi = inputs[l].detach().numpy()
        imgi_ndvi = imgi[12:14,:]
        imgi_lswi = imgi[14,:]
        imgi = imgi[0:12,:]
        # print(imgi.size())

        # imgo = outputs[l].permute(1,2,0)
        # print(imgo.size())
        #imgi = imgi[:,:,2:5]
        imgo = outputs[l].detach().numpy()
        imgo_ndvi = imgo[12:14,:]
        imgo_lswi = imgo[14,:]
        imgo = imgo[0:12,:]

        img_diff = imgo - imgi
        error = np.sum(np.power(img_diff,2))

        img_diff = img_diff/imgi
        # img_diff =  img_diff / img_diff.max()

        scatter_spectrums([imgi,imgo],labels[l]*500)
        # plot_spectrum(imgo,labels[l]*500)

        scatter_spectrums([imgi_ndvi,imgo_ndvi],labels[l]*500)
        # plot_spectrum(imgo_ndvi,labels[l]*500)
        scatter_spectrum(img_diff,labels[l]*500)
        print(error)

        # fig, axs = plt.subplots(1,3, figsize=(10, 6))
        # fig = plt.gcf()
        # fig.set_size_inches(18.5,10.5)
       
        # img_show = np.array(imgi[:,:,0:3])
        # # m = img_show.max()
        # # img_show = img_show / m
        
        # img_show2 = np.array(imgo[:,:,0:3].detach().numpy())
        # # m = img_show2.max()
        # # img_show2 = img_show2 / m

        # axs[0].imshow(img_show/img_show.max())
        # axs[1].imshow(img_show2/img_show.max())
        # img_diff = (img_show2 - img_show)
        # axs[2].imshow(img_diff/img_diff.max())
        # error = np.sum(np.power(img_diff,2))
        # a = img_diff/img_show
        # axs[3].imshow(a)

        # print(error)
        # # axs[2].title(str(error))
        # plt.show()


      
    break

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from visualization import plot_spectrum
total_loss = 0
total_samples = 0
for data in test_loader:
    inputs, labels = data
    # outputs =conv_deconv(inputs)
    features, pool_size = conv(inputs)
    outputs = deconv(features, pool_size )
      
    for l in range(5):

        # imgi = inputs[l].permute(1,2,0) 
        imgi = inputs[l].detach().numpy()
        # print(imgi.size())

        # imgo = outputs[l].permute(1,2,0)
        # print(imgo.size())
        #imgi = imgi[:,:,2:5]
        imgo = outputs[l].detach().numpy()

        plot_spectrum(imgi,labels[l]*500)
        plot_spectrum(imgo,labels[l]*500)

        img_diff = imgo - imgi
        error = np.sum(np.power(img_diff,2))

        img_diff = img_diff/imgi
        # img_diff =  img_diff / img_diff.max()

        plot_spectrum(img_diff,labels[l]*500)
        print(error)

        # fig, axs = plt.subplots(1,3, figsize=(10, 6))
        # fig = plt.gcf()
        # fig.set_size_inches(18.5,10.5)
       
        # img_show = np.array(imgi[:,:,0:3])
        # # m = img_show.max()
        # # img_show = img_show / m
        
        # img_show2 = np.array(imgo[:,:,0:3].detach().numpy())
        # # m = img_show2.max()
        # # img_show2 = img_show2 / m

        # axs[0].imshow(img_show/img_show.max())
        # axs[1].imshow(img_show2/img_show.max())
        # img_diff = (img_show2 - img_show)
        # axs[2].imshow(img_diff/img_diff.max())
        # error = np.sum(np.power(img_diff,2))
        # a = img_diff/img_show
        # axs[3].imshow(a)

        # print(error)
        # # axs[2].title(str(error))
        # plt.show()


      
    break